In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

In [ ]:
class Churnpredict:
    def __init__(self,X,y):
        self.X=X
        self.y=y
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
df=pd.read_csv('./final_dataset.csv')

X=df.drop(columns=['Churn']).values
y=df['Churn'].values

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

batch_size=50
learning_rate=0.001
dataset = Churnpredict(X, y)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size  # This ensures the split sums up to the total dataset length

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])


train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

examples = iter(train_loader)
sample, labels = next(examples)

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self,input_size, hidden_size):
        super(NeuralNet,self).__init__()
        self.l1=nn.Linear(input_size,hidden_size)
        self.relu=nn.ReLU()
        self.l2=nn.Linear(hidden_size,hidden_size*2)
        self.relu2=nn.ReLU()
        self.l3=nn.Linear(hidden_size*2,hidden_size)
        self.relu3=nn.ReLU()
        self.l4=nn.Linear(hidden_size,1)
        self.sigmoid=nn.Sigmoid()

    def forward(self,x):
        out=self.l1(x)
        out=self.relu(out)
        out=self.l2(out)
        out=self.relu2(out)
        out=self.l3(out)
        out=self.relu3(out)
        out=self.l4(out)
        out=self.sigmoid(out)

        return out

In [ ]:
input_size=6
hidden_size = 150
num_classes=1

model=NeuralNet(input_size=input_size,hidden_size=hidden_size)
model = model.to(device)

criterion=nn.BCELoss()
optimiser=torch.optim.Adam(model.parameters(),lr=learning_rate)

n_total_steps = len(train_loader)

num_epochs=100

for epoch in range(num_epochs):
    for i, (sample, labels) in enumerate(train_loader):
        sample=sample.to(device)
        labels = labels.to(device).view(-1, 1)

        # forward
        outputs = model(sample).to(device)
        loss = criterion(outputs, labels)

        # backward
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        if (i + 1) % 100 == 0:
            print(
                f"epcoh{epoch + 1}/{num_epochs}, step: {i + 1}/{n_total_steps} loss: {loss.item():.4f}"
            )


In [ ]:
with torch.no_grad():
    n_correct=0
    n_samples=0

    for sample,labels in test_loader:
        sample=sample.to(device)
        labels=labels.to(device)

        output=model(sample)
        #value,index
        _,predictions=torch.max(output,1)
        n_samples +=labels.shape[0]
        n_correct +=(predictions==labels).sum().item()
acc=100.0 * (n_correct/n_samples)
print(f'Accuracy: {acc}')

In [ ]:
from sklearn.metrics import roc_auc_score

# Put model in evaluation mode
model.eval()

y_true = []
y_scores = []

with torch.no_grad():
    for sample, labels in test_loader:
        sample = sample.to(device)
        labels = labels.to(device)

        outputs = model(sample).cpu().numpy()  # Convert to NumPy
        labels = labels.cpu().numpy()

        y_scores.extend(outputs.flatten())  # Store predictions
        y_true.extend(labels)  # Store true labels

# Compute AUC-ROC
auc_score = roc_auc_score(y_true, y_scores)
print(f"AUC-ROC Score: {auc_score:.4f}")


In [ ]:
import torch
torch.save(model, "churn_model.pkl")
print("Model saved as churn_model.pkl")
